## LIBRARY

In [1]:
import os
import re
import pandas as pd
import tkinter as tk
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict
import math
from collections import Counter
import mplcursors
import matplotlib as mpl
import textwrap
import numpy as np
from tkinter import IntVar
from upsetplot import UpSet
from pyvis.network import Network
from sklearn.preprocessing import MinMaxScaler

In [2]:

directory = 'C:/Users/begon/Desktop/LMI PROJECT/GIT-LMI/deneme1' 

pattern = re.compile(r'(\"resourceType\": \"(Patient|Encounter|Condition|Observation|Procedure|Medication)\")')

classes = ['Patient', 'Encounter', 'Condition', 'Observation', 'Procedure', 'Medication']

current_resource_type = None
new_resource_type = None
resource_type = None
current_id = None
new_id = None
current_references = []
current_codes = []
display = []
current_patient_id = None
previous_display = None 
previous_code = None 
code_starts_with_LA = False  # Flag to track if the current code starts with "LA"


data = {'ResourceType': [], 'ID': [], 'References': [], 'Codes': [], 'PatientID': []}

counts = {resource_type: 1 for resource_type in classes}

INIT = False

for filename in os.listdir(directory):
    if filename.endswith('.json'):
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as f:
            for line in f:
                match = pattern.search(line)
                if '"resourceType": "' in line and not match:
                    resource_type = line.split('"')[3]
                    INIT = False
                if match:
                    if match.group(2) in classes:
                        Let = match.group(2)[:2]
                        #print("resource_type", Let)
                        new_resource_type = match.group(2)
                        INIT = True
                        next_line = next(f, None)
                        if next_line and '"id": "' in next_line:
                            new_id = next_line.split('"id": "')[1].split('"')[0]
                if current_resource_type == new_resource_type and '"reference": "urn:uuid:' in line and INIT:
                    reference = line.split(':')[3].split('"')[0]
                    current_references.append(reference)    
                    
                if current_resource_type == new_resource_type and '"code": "' in line and INIT:
                    code = line.split('"code": "')[1].split('"')[0]
                    if any(char.isdigit() for char in code):
                        #print(current_patient_id, code)
                        next_line = next(f, None)
                        if next_line and '"display": "' in next_line:
                            if code.startswith("LA") and previous_display is not None:                    
                                display = previous_display + ': ' + next_line.split('"display": "')[1].split('"')[0]
                                code = previous_code
                                #current_codes[-1] = (previous_code, display)
                                current_codes[-1] = (Let + previous_code, display)
                                code_starts_with_LA = True
                            else:
                                display = next_line.split('"display": "')[1].split('"')[0]
                                #current_codes.append((code, display))
                                current_codes.append((Let + code, display))
                                code_starts_with_LA = False
                                #print('(code, display)', (current_patient_id, code, display))
                        # Update previous display only if the code doesn't start withp "LA"
                        if not code_starts_with_LA:
                            previous_display = display
                            previous_code = code

                 
                if current_resource_type != new_resource_type  and INIT:
                    if current_resource_type and current_id:
                        data['ResourceType'].append(current_resource_type)
                        data['ID'].append(current_id)
                        if current_resource_type == 'Patient':
                            current_patient_id = current_id
                        data['References'].append(tuple(current_references))
                        data['Codes'].append(tuple(current_codes))
                        data['PatientID'].append(current_patient_id)
                        current_references = []
                        current_codes = []
                    current_resource_type = new_resource_type    
                    current_id = new_id

df = pd.DataFrame(data)


In [3]:
flat_df = df.explode('Codes').reset_index().iloc[:,[1,4,5]]
Codes = [code[0] for code in flat_df['Codes']]
Displays = [display[1] for display in flat_df['Codes']]
flat_df['Codes'] = Codes
flat_df['Displays'] = Displays
flat_df = flat_df[['PatientID', 'Codes', 'Displays', 'ResourceType']]

try_df = flat_df#.iloc[:100,:]
try_df

# icd_patient = try_df.pivot_table(index='PatientID', columns='Codes', aggfunc='size', fill_value=0)
# icd_patient = icd_patient.loc[:, (icd_patient != 0).any(axis=0)]

# # Co-Occurrence matrix
# co_occurrence_matrix = icd_patient.T.dot(icd_patient)
# print(co_occurrence_matrix)
# np.fill_diagonal(co_occurrence_matrix.values, 0)
# # Create graph
# graph = nx.from_pandas_adjacency(co_occurrence_matrix)  

,PatientID,Codes,Displays,ResourceType
0,0a130e19-fd58-faa6-bbab-f91e6c9b31b4,Pa2106-3,White,Patient
1,0a130e19-fd58-faa6-bbab-f91e6c9b31b4,Pa2186-5,Not Hispanic or Latino,Patient
2,0a130e19-fd58-faa6-bbab-f91e6c9b31b4,En185347001,Encounter for problem,Encounter
3,0a130e19-fd58-faa6-bbab-f91e6c9b31b4,En735971005,Fish (substance),Encounter
4,0a130e19-fd58-faa6-bbab-f91e6c9b31b4,En185347001,Encounter for problem,Encounter
...,...,...,...,...
17579,44fe29bb-b0fe-1542-29c9-741ab4be569a,Ob93035-4,"At any point in the past 2 years, has season o...",Observation
17580,44fe29bb-b0fe-1542-29c9-741ab4be569a,Ob32624-9,Race: White,Observation
17581,44fe29bb-b0fe-1542-29c9-741ab4be569a,Ob56051-6,Do you consider yourself Hispanic/Latino?: No,Observation
17582,44fe29bb-b0fe-1542-29c9-741ab4be569a,Ob55758-7,Patient Health Questionnaire 2 item (PHQ-2) to...,Observation


In [4]:
en_df = flat_df[flat_df['ResourceType'] == 'Encounter']
co_df = flat_df[flat_df['ResourceType'] == 'Condition']
ob_df = flat_df[flat_df['ResourceType'] == 'Observation']
pr_df = flat_df[flat_df['ResourceType'] == 'Procedure']
me_df = flat_df[flat_df['ResourceType'] == 'Medication']



In [5]:
# List of resources
resources = [try_df, en_df, co_df, ob_df, pr_df, me_df]

# Dictionary to store co-occurrence matrices for each resource
co_occurrence_matrices = {}

for r in resources:
    # Transpose data
    icd_patient = r.pivot_table(index='PatientID', columns='Codes', aggfunc='size', fill_value=0)
    icd_patient = icd_patient.loc[:, (icd_patient != 0).any(axis=0)]

    # Co-Occurrence matrix
    co_occurrence_matrix = icd_patient.T.dot(icd_patient)
    np.fill_diagonal(co_occurrence_matrix.values, 0)

    # Store co-occurrence matrix based on resource type
    if r.equals(try_df):
        co_occurrence_matrices['Main'] = co_occurrence_matrix
    elif r.equals(en_df):
        co_occurrence_matrices['Encounter'] = co_occurrence_matrix
    elif r.equals(co_df):
        co_occurrence_matrices['Condition'] = co_occurrence_matrix
    elif r.equals(ob_df):
        co_occurrence_matrices['Observation'] = co_occurrence_matrix
    elif r.equals(pr_df):
        co_occurrence_matrices['Procedure'] = co_occurrence_matrix
    elif r.equals(me_df):
        co_occurrence_matrices['Medication'] = co_occurrence_matrix


graph = nx.from_pandas_adjacency(co_occurrence_matrices['Main'])

# ADD COLOR AND LABEL

In [6]:
import tkinter as tk
from tkinter import ttk, messagebox
import networkx as nx
import tempfile
import webview
import os
import pandas as pd
import numpy as np
from pyvis.network import Network

# Define colors for each subgroup
SUBGROUP_COLORS = { #"#00bfff", "#ffc0cb", "#3da831", "#9a31a8", dark blue"#3155a8", "#eb4034"
    'Condition': "#00bfff",
    'Encounter': "#3da831",
    'Observation': "#ffc0cb",
    'Procedure': "#9a31a8",
    'Medication': "#eb4034"
}

# Define global variables
selected_node = None
selected_var = None
PADDING = 0.1
n = None
canvas = None
num_nodes_to_visualize = 0
webview_window = None

def get_display_label(try_df, node_code):
    """Get the display label for a given node code from the DataFrame."""
    label = try_df.loc[try_df['Codes'] == node_code, 'Displays']
    return label.values[0] if not label.empty else node_code

def wrap_text(text, max_width=15):
    """Wrap text to fit within a maximum width, defaulting to 15 characters."""
    return "\n".join(textwrap.wrap(text, width=max_width))


# Function to generate the graph
def generate_graph(graph, num_nodes_to_visualize, selected_code, co_occurrence_matrix, show_labels):
    global webview_window
    
    if selected_code:
        print(f"Selected code: {selected_code}")

        # Initialize a PyVis network
        net = Network(notebook=True, cdn_resources='remote')

        # Get the main co-occurrence matrix
        main_df = co_occurrence_matrix['Main']

        # Sort neighbors by descending order
        neighbors_sorted = main_df.loc[selected_code].sort_values(ascending=False)
        top_neighbors = list(neighbors_sorted.index[:15])
        print(f"Top neighbors: {top_neighbors}")


        # Function to add nodes and edges to the PyVis network
        def add_nodes_edges(graph, child_df, prefix, group_name):
            top_neighbor = None
            for neighbor_code in neighbors_sorted.index:
                if neighbor_code.startswith(prefix):
                    top_neighbor = neighbor_code
                    break
            print('top_neighbor', top_neighbor)

            if top_neighbor:
                # Add labels
                selected_code_label = get_display_label(try_df, selected_code)
                top_neighbor_label = get_display_label(try_df, top_neighbor)
                
                # Add nodes for selected code and top neighbor
                net.add_node(selected_code, title=selected_code, label= wrap_text(selected_code_label) if show_labels else selected_code[2:], color=SUBGROUP_COLORS.get(group_name, 'gray'))
                net.add_node(top_neighbor, title=top_neighbor, label= wrap_text(top_neighbor_label) if show_labels else top_neighbor[2:], color=SUBGROUP_COLORS.get(group_name, 'gray'))

                # Add edge between selected code and top neighbor
                net.add_edge(selected_code, top_neighbor, value=int(main_df.loc[selected_code, top_neighbor]))

                # Get the top 3 neighbors of the top neighbor (excluding itself)
                top_neighbor_row = child_df.loc[top_neighbor].sort_values(ascending=False)
                top_neighbors = list(top_neighbor_row.index[:num_nodes_to_visualize])

                # Add nodes and edges for the top neighbors of the top neighbor
                for neighbor in top_neighbors:
                    if neighbor != top_neighbor and child_df.loc[top_neighbor, neighbor] > 0:
                        neighbor_label = get_display_label(try_df, neighbor)
                        net.add_node(neighbor, title=neighbor, label= wrap_text(neighbor_label) if show_labels else neighbor[2:], color=SUBGROUP_COLORS.get(group_name, 'gray'))
                        net.add_edge(top_neighbor, neighbor, value=int(child_df.loc[top_neighbor, neighbor]))

                        # Add edges from the top neighbor to its neighbors
                        for other_code in top_neighbor_row.index[:num_nodes_to_visualize]:
                            if other_code != neighbor and child_df.loc[neighbor, other_code] > 0:
                                other_code_label = get_display_label(try_df, other_code)
                                net.add_node(other_code, title=other_code, label= wrap_text(other_code_label) if show_labels else other_code[2:], color=SUBGROUP_COLORS.get(group_name, 'gray'))
                                net.add_edge(neighbor, other_code, value=int(child_df.loc[neighbor, other_code]))
                      
                             
        # Add nodes and edges for each subgroup
        if 'Condition' in co_occurrence_matrix:
            add_nodes_edges(net, co_occurrence_matrix['Condition'], 'Co', 'Condition')
        if 'Encounter' in co_occurrence_matrix:
            add_nodes_edges(net, co_occurrence_matrix['Encounter'], 'En', 'Encounter')
        if 'Observation' in co_occurrence_matrix:
            add_nodes_edges(net, co_occurrence_matrix['Observation'], 'Ob', 'Observation')
        if 'Procedure' in co_occurrence_matrix:
            add_nodes_edges(net, co_occurrence_matrix['Procedure'], 'Pr', 'Procedure')
        if 'Medication' in co_occurrence_matrix:
            add_nodes_edges(net, co_occurrence_matrix['Medication'], 'Me', 'Medication')

        # Show node titles in the network (optional)
        net.show_buttons()#filter_=['nodes'])

        # Temporary file to save the HTML visualization
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.html')
        temp_file_name = temp_file.name
        temp_file.close()

        # Save the PyVis network to HTML
        net.show(temp_file_name)

        # Destroy the previous webview window if it exists
        if webview_window:
            webview_window.destroy()
            webview_window = None

        # Open the HTML file in a new webview window
        webview_window = webview.create_window("PyVis Graph", url=f"file://{os.path.abspath(temp_file_name)}")
        webview.start()

    else:
        # Display message box if no code is selected
        messagebox.showinfo("Info", "Please select a code first.")

# Tkinter GUI setup
def open_graph_window():
    global num_nodes_to_visualize

    slider_label = tk.Label(root, text="Select the number of nodes to visualize:")
    slider_label.grid(row=1, column=0, padx=10, pady=10)

    num_nodes_to_visualize = tk.IntVar(value=1)
    slider = tk.Scale(root, from_=1, to=10, orient=tk.HORIZONTAL, variable=num_nodes_to_visualize)
    slider.grid(row=2, column=0, padx=10, pady=10)

    generate_button = tk.Button(root, text="Generate Graph", command=lambda: generate_graph(graph, num_nodes_to_visualize.get(), selected_code, co_occurrence_matrices, show_labels_var.get()))
    generate_button.grid(row=3, column=0, padx=10, pady=10)

def on_dropdown_select(event):
    global selected_code
    #selected_code = dropdown.get()
    selected_index = dropdown.current()  # Get the index of the selected item
    selected_code = codes[selected_index]
    print(f"Dropdown selected code: {selected_code}")

# Create the main Tkinter window
root = tk.Tk()
root.title("CO-OCCURANCES IN FHIR CODES")


# Define the selected groups and dropdown menu (assuming `codes` is defined globally)
codes = list(co_occurrence_matrices['Main'].columns)
shortened_codes = [code[2:] for code in codes]
selected_groups = []
dropdown = ttk.Combobox(root, values=shortened_codes, state="readonly")
dropdown.grid(row=0, column=0, padx=10, pady=10)
dropdown.bind("<<ComboboxSelected>>", on_dropdown_select)


# Checkbox to show or hide labels
show_labels_var = tk.BooleanVar()
show_labels_checkbox = tk.Checkbutton(root, text="Show Labels", variable=show_labels_var)
show_labels_checkbox.grid(row=0, column=1, padx=10, pady=10)


open_graph_window()

root.mainloop()

Dropdown selected code: Co105531004
Selected code: Co105531004
Top neighbors: ['Ob71802-3', 'Pr410401003', 'En162673000', 'Ob29463-7', 'Ob8462-4', 'Ob8480-6', 'Ob85354-9', 'Ob8867-4', 'Ob9279-1', 'Ob93027-1', 'Ob93025-5', 'Ob93026-3', 'Ob266919005', 'Ob93028-9', 'Ob93029-7']
top_neighbor Co314529007
top_neighbor En162673000
top_neighbor Ob71802-3
top_neighbor Pr410401003
top_neighbor Me807283
C:\Users\begon\AppData\Local\Temp\tmp1o0c7zse.html
Selected code: Co105531004
Top neighbors: ['Ob71802-3', 'Pr410401003', 'En162673000', 'Ob29463-7', 'Ob8462-4', 'Ob8480-6', 'Ob85354-9', 'Ob8867-4', 'Ob9279-1', 'Ob93027-1', 'Ob93025-5', 'Ob93026-3', 'Ob266919005', 'Ob93028-9', 'Ob93029-7']
top_neighbor Co314529007
top_neighbor En162673000
top_neighbor Ob71802-3
top_neighbor Pr410401003
top_neighbor Me807283
C:\Users\begon\AppData\Local\Temp\tmpb0evdc20.html
